## Section 6 - Create a Search Service
Create a Search Service to search for all unstructured data

This is a reminder, we have already processed unstructured data from the analyst reports

In [ ]:
SELECT * FROM DOCUMENT_AI.ANALYST_REPORTS_CHUNKED;

We have also transcribed the earnings calls, and grouped them together to form larger chunks

In [ ]:
SELECT * FROM DEFAULT_SCHEMA.summary_text;

Let's now create a view which standardised the results in order to combine with the analyst reports.  A document type has also been added for deterministic type of filtering where necessary.

In [ ]:
CREATE VIEW IF NOT EXISTS DEFAULT_SCHEMA.EARNINGS_CALLS_FORMATTED AS 
SELECT RELATIVE_PATH, 'NA' RATING, 'NA' DATE_REPORT, 'NA' NAME_OF_REPORT_PROVIDER,  ['EARNINGS_CALLS'] DOCUMENT_TYPE,RELATIVE_PATH DOCUMENT, 
TEXT,REPLACE(REPLACE(REPLACE(REPLACE(RELATIVE_PATH,'EARNINGS_',''),'.mp3',''),'_',''),'FY20','FY')::ARRAY PERIOD, SENTIMENT FROM DEFAULT_SCHEMA.summary_text;

SELECT * FROM DEFAULT_SCHEMA.EARNINGS_CALLS_FORMATTED limit 10

Next, you will union the earnings calls with the analyst reports

In [ ]:
CREATE TABLE IF NOT EXISTS DEFAULT_SCHEMA.TEXT_AND_SOUND AS 

select * from DEFAULT_SCHEMA.EARNINGS_CALLS_FORMATTED

UNION ALL

SELECT * FROM DOCUMENT_AI.ANALYST_REPORTS_CHUNKED

### CREATE A SEARCH SERVICE
Now, let's create a search service

In [ ]:
CREATE OR REPLACE  CORTEX SEARCH SERVICE DEFAULT_SCHEMA.CHUNKED_REPORTS
  ON TEXT
  ATTRIBUTES DOCUMENT_TYPE,PERIOD
  WAREHOUSE = DEFAULT_WH
  TARGET_LAG = '1 hour'
  COMMENT = 'SEARCH SERVICE FOR REPORTS - CHUNKED'
  AS SELECT * FROM DEFAULT_SCHEMA.TEXT_AND_SOUND;

In [ ]:
USE SCHEMA DEFAULT_SCHEMA;
SHOW CORTEX SEARCH SERVICES

Now you have created the search service, you can now go to **Projects > Streamlits > CORTEX_SEARCH** to see the search tool in action.